In [2]:
import pandas as pd
# Load your data
data = pd.read_csv('Opioid Prediction/training_healthcare_opioid_prediction_data.csv')
data.head(1)

,PatientID,Age,Gender,Race,ZipCode,ChronicPainConditions,NumOpioidPrescriptions,AverageDosage,DurationOfPrescriptions,NumHealthcareVisits,...,Refills,MedicationClass,Adherence,ClinicalNotes,Specialty,AppointmentType,SubSpecialty,TimeofAppointment,TimeSeenbyPhysician,TotalTimeSpentwithPhysician
0,4479647884230,54,Female,Other,73301,Fibromyalgia,6,80,7,18,...,0,Opioid,Moderate,Post-operative pain managed with Hydrocodone.,Orthopedics,Routine Check-up,Specialized,13:39:02,16:33:59,36


In [8]:
categorical_columns = [
    'Gender', 'Race', 'ChronicPainConditions', 'PainManagementTreatment',
    'MedicationName', 'Dosage', 'Frequency',
    'MedicationClass', 'Adherence', 'ClinicalNotes', 'Specialty',
    'AppointmentType', 'SubSpecialty',
]
numerical_columns = [
    'PatientID', 'Age', 'ZipCode', 'NumOpioidPrescriptions',
    'AverageDosage', 'DurationOfPrescriptions', 'NumHealthcareVisits',
    'NumHospitalizations', 'Duration', 'Refills',
    'TotalTimeSpentwithPhysician'
]
# Get min and max values for numerical columns
numerical_min_max = data[numerical_columns].agg(['min', 'max'])

# Get unique classes for categorical columns
categorical_classes = {col: data[col].unique() for col in categorical_columns}

# Print the results
print("Numerical columns min and max values:")
print(numerical_min_max)

Numerical columns min and max values:
         PatientID  Age  ZipCode  NumOpioidPrescriptions  AverageDosage  \
min    25758856396   18    10001                       1              5   
max  9984733852024   79    90210                      19             99   

     DurationOfPrescriptions  NumHealthcareVisits  NumHospitalizations  \
min                        1                    0                    0   
max                       29                   19                    4   

     Duration  Refills  TotalTimeSpentwithPhysician  
min         1        0                           10  
max        29        4                           59  


In [9]:
print("\nCategorical columns unique classes:")
print(categorical_classes)


Categorical columns unique classes:
{'Gender': array(['Female', 'Male'], dtype=object), 'Race': array(['Other', 'Asian', 'White', 'Black', 'Hispanic'], dtype=object), 'ChronicPainConditions': array(['Fibromyalgia', 'Cancer Pain', 'Post-Surgery Pain',
       'Chronic Back Pain', 'Arthritis'], dtype=object), 'PainManagementTreatment': array(['No', 'Yes'], dtype=object), 'MedicationName': array(['Hydrocodone', 'Hydromorphone', 'Oxymorphone', 'Tramadol',
       'Codeine', 'Morphine', 'Buprenorphine', 'Oxycodone', 'Meperidine',
       'Methadone', 'Fentanyl', 'Tapentadol'], dtype=object), 'Dosage': array(['10 mg', '100 mcg/hour', '20 mg', '80 mg', '60 mg',
       '12.5 mcg/hour', '50 mcg/hour', '40 mg', '2.5 mg', '75 mcg/hour',
       '5 mg', '30 mg'], dtype=object), 'Frequency': array(['every 8 hours', 'every 4-6 hours', 'every 12 hours', 'once daily'],
      dtype=object), 'MedicationClass': array(['Opioid', 'Narcotic', 'Analgesic'], dtype=object), 'Adherence': array(['Moderate', 'Low', 

# Creating Target Columns

### Numerical columns:
- Age: Age can be a factor in opioid addiction risk.
- NumOpioidPrescriptions: Directly related to opioid usage.
- AverageDosage: High dosages may indicate higher risk.
- DurationOfPrescriptions: Longer durations may suggest higher risk.
- NumHealthcareVisits: Frequent healthcare visits could indicate underlying issues.
- NumHospitalizations: Multiple hospitalizations might correlate with higher risk.
- Refills: More refills can indicate dependency.
- TotalTimeSpentwithPhysician: Indicates the level of care and monitoring.

### Categorical columns:

- Gender: There could be gender-related differences in opioid addiction.
- Race: Racial differences may exist in opioid addiction risk.
- ChronicPainConditions: Chronic pain is a major reason for opioid prescriptions.
- PainManagementTreatment: Indicates type of pain management, which is relevant.
- MedicationName: Specific medications may have different risk profiles.
- Dosage: Helps understand the prescribed dosage pattern.
- Frequency: Frequency of medication intake.
- MedicationClass: Class of medication could be relevant.
- Adherence: Indicates whether the patient follows prescribed treatment.
- ClinicalNotesL column can potentially be relevant.
- Specialty: Specialty of the prescribing doctor.
- AppointmentType: Type of appointment may give context to prescription.
- SubSpecialty: Additional context on the prescribing doctor's expertise.

### Date and time columns:

- TimeofAppointment, TimeSeenbyPhysician, PrescriptionDate: These columns might not be directly useful for predicting risk but could provide temporal context if engineered into features like duration between appointments, etc.

#### We can make this target variable binary(low, High), Multiclass Classification(low, moderate, High) and percentage wise(Regression)
### Highly Risk Factor : 
- High number of opioid prescriptions and high average dosage
- Multiple hospitalizations
- Long duration of prescriptions
- Low or Moderate adherence to treatment
- Multiple refills


### Apprach 1 to create Target Columns
- Use Simple NumOpioidPrescriptions > 10 then high_risk_threshold

### Apprach 2 to create Target Columns
- Combination of many Variables 
- NumOpioidPrescriptions > 10 & AverageDosage > 50
- NumHospitalizations > 2
- DurationOfPrescriptions' > 20
- Adherence wiht 'Low', 'Moderate'
- Refills >= 2

### Approach 3 to create Target Columns
- By adding weights of column(Based on testing) (Complex)

### Approach 4 to create Target Columns
- By Medication name with overdose (Complex)

In [2]:
# data['Target'] = (data['NumOpioidPrescriptions'] > 10).astype(int)
# print(data['Target'].value_counts())
# data.head()


import pandas as pd

def create_target_variable(df):

  # Risk factors and adjustments based on domain knowledge (replace placeholders with your insights)
  high_risk_medications = ['Fentanyl', 'Methadone', 'Morphine']  # Potentially higher risk opioids
  long_durations = df['DurationOfPrescriptions'] >= 30  # Consider prescriptions lasting over a month high risk
  frequent_refills = df['Refills'] >= 3  # Frequent refills could suggest potential misuse
  low_adherence = df['Adherence'] == 'Low'  # Low adherence might increase risk
  previous_abuse = df['ClinicalNotes'].str.contains('substance abuse', case=False)  # Text analysis for prior abuse mentions

  # Calculate a preliminary risk score (adapt based on your analysis)
  df['RiskScore'] = df['NumOpioidPrescriptions'] * df['AverageDosage']
  df.loc[(df['MedicationName'].isin(high_risk_medications)) | long_durations | frequent_refills | low_adherence | previous_abuse, 'RiskScore'] *= 2  # Double risk score for high-risk factors

  # Define risk categories based on RiskScore (adjust thresholds as needed)
  df['RiskLevel'] = pd.cut(df['RiskScore'], 3, labels=['Low', 'Medium', 'High'])

  # Target variable for risk level
  # df['Target_RiskLevel'] = df['RiskLevel']

  print("Target variable 'Target_RiskLevel' created based on a RiskScore incorporating domain knowledge.")
  return df

# Create the target variable
data = create_target_variable(data.copy())  # Operate on a copy to avoid modifying the original data
data.head()

Target variable 'Target_RiskLevel' created based on a RiskScore incorporating domain knowledge.


,PatientID,Age,Gender,Race,ZipCode,ChronicPainConditions,NumOpioidPrescriptions,AverageDosage,DurationOfPrescriptions,NumHealthcareVisits,...,Adherence,ClinicalNotes,Specialty,AppointmentType,SubSpecialty,TimeofAppointment,TimeSeenbyPhysician,TotalTimeSpentwithPhysician,RiskScore,RiskLevel
0,4479647884230,54,Female,Other,73301,Fibromyalgia,6,80,7,18,...,Moderate,Post-operative pain managed with Hydrocodone.,Orthopedics,Routine Check-up,Specialized,13:39:02,16:33:59,36,480,Low
1,6354629782347,21,Female,Asian,60601,Cancer Pain,11,70,25,12,...,Low,Prescribed Oxymorphone for severe pain.,Pain Management,Consultation,General,19:44:46,17:33:37,51,1540,Medium
2,4665077438222,67,Female,White,90210,Fibromyalgia,5,15,7,10,...,Moderate,Patient reports effective pain relief with Tap...,Oncology,Routine Check-up,Specialized,18:05:34,04:39:28,46,75,Low
3,2844761285195,41,Male,White,10001,Fibromyalgia,16,94,19,14,...,Low,Using Tramadol for moderate pain management.,Orthopedics,Consultation,Specialized,15:16:11,15:52:51,16,3008,High
4,4785391177871,36,Male,White,10001,Post-Surgery Pain,7,22,6,9,...,High,Patient reports effective pain relief with Tap...,Oncology,Follow-up,General,22:17:05,02:17:33,14,154,Low


In [3]:
# Display all columns
print(data.columns)

Index(['PatientID', 'Age', 'Gender', 'Race', 'ZipCode',
       'ChronicPainConditions', 'NumOpioidPrescriptions', 'AverageDosage',
       'DurationOfPrescriptions', 'NumHealthcareVisits', 'NumHospitalizations',
       'PainManagementTreatment', 'PrescriptionDate', 'MedicationName',
       'Dosage', 'Frequency', 'Duration', 'Refills', 'MedicationClass',
       'Adherence', 'ClinicalNotes', 'Specialty', 'AppointmentType',
       'SubSpecialty', 'TimeofAppointment', 'TimeSeenbyPhysician',
       'TotalTimeSpentwithPhysician', 'RiskScore', 'RiskLevel'],
      dtype='object')


### Not Relevant Columns
- patientID: Typically used for identification purposes
- PrescriptionDate: Might not directly impact risk prediction unless temporal patterns are crucial.
- TimeofAppointment, TimeSeenbyPhysician: Unless time-related patterns (e.g., appointment frequency) are critical, these may not be necessary.
- TotalTimeSpentwithPhysician: Relevant if indicating consultation intensity related to opioid prescription.
- ZipCode: Could be relevant for geographic patterns
- AppointmentType: May provide context on the nature of medical encounters.
- Specialty, SubSpecialty: Relevant if specific medical expertise impacts opioid prescription practices.
- ClinicalNotes: Could contain specific medical details but may require extensive text processing for meaningful insights.

In [4]:
columns_to_remove = ['PatientID', 'PrescriptionDate', 'ClinicalNotes', 
                     'TimeofAppointment', 'TimeSeenbyPhysician', 
                     'TotalTimeSpentwithPhysician', 'ZipCode', 
                     'Specialty', 'SubSpecialty']

data = data.drop(columns=columns_to_remove)
data.head()

,Age,Gender,Race,ChronicPainConditions,NumOpioidPrescriptions,AverageDosage,DurationOfPrescriptions,NumHealthcareVisits,NumHospitalizations,PainManagementTreatment,MedicationName,Dosage,Frequency,Duration,Refills,MedicationClass,Adherence,AppointmentType,RiskScore,RiskLevel
0,54,Female,Other,Fibromyalgia,6,80,7,18,0,No,Hydrocodone,10 mg,every 8 hours,1,0,Opioid,Moderate,Routine Check-up,480,Low
1,21,Female,Asian,Cancer Pain,11,70,25,12,0,Yes,Hydromorphone,100 mcg/hour,every 8 hours,20,0,Narcotic,Low,Consultation,1540,Medium
2,67,Female,White,Fibromyalgia,5,15,7,10,2,No,Hydrocodone,20 mg,every 4-6 hours,7,2,Analgesic,Moderate,Routine Check-up,75,Low
3,41,Male,White,Fibromyalgia,16,94,19,14,2,No,Oxymorphone,10 mg,every 12 hours,28,0,Analgesic,Low,Consultation,3008,High
4,36,Male,White,Post-Surgery Pain,7,22,6,9,3,Yes,Tramadol,80 mg,every 4-6 hours,25,2,Analgesic,High,Follow-up,154,Low


In [5]:
import pandas as pd
# Select categorical columns
categorical_columns = data.select_dtypes(include=['object', 'category']).columns
# Print unique values for each column
for column in categorical_columns:
    unique_values = data[column].unique()
    print(f"Column: {column}")
    print(unique_values)
    print("\n")

Column: Gender
['Female' 'Male']


Column: Race
['Other' 'Asian' 'White' 'Black' 'Hispanic']


Column: ChronicPainConditions
['Fibromyalgia' 'Cancer Pain' 'Post-Surgery Pain' 'Chronic Back Pain'
 'Arthritis']


Column: PainManagementTreatment
['No' 'Yes']


Column: MedicationName
['Hydrocodone' 'Hydromorphone' 'Oxymorphone' 'Tramadol' 'Codeine'
 'Morphine' 'Buprenorphine' 'Oxycodone' 'Meperidine' 'Methadone'
 'Fentanyl' 'Tapentadol']


Column: Dosage
['10 mg' '100 mcg/hour' '20 mg' '80 mg' '60 mg' '12.5 mcg/hour'
 '50 mcg/hour' '40 mg' '2.5 mg' '75 mcg/hour' '5 mg' '30 mg']


Column: Frequency
['every 8 hours' 'every 4-6 hours' 'every 12 hours' 'once daily']


Column: MedicationClass
['Opioid' 'Narcotic' 'Analgesic']


Column: Adherence
['Moderate' 'Low' 'High']


Column: AppointmentType
['Routine Check-up' 'Consultation' 'Follow-up']


Column: RiskLevel
['Low', 'Medium', 'High']
Categories (3, object): ['Low' < 'Medium' < 'High']




In [6]:
data["Dosage_numeric"] = data["Dosage"].str.extract("(\d+\.?\d*)", expand=False)
del data["Dosage"]  
data.head()

,Age,Gender,Race,ChronicPainConditions,NumOpioidPrescriptions,AverageDosage,DurationOfPrescriptions,NumHealthcareVisits,NumHospitalizations,PainManagementTreatment,MedicationName,Frequency,Duration,Refills,MedicationClass,Adherence,AppointmentType,RiskScore,RiskLevel,Dosage_numeric
0,54,Female,Other,Fibromyalgia,6,80,7,18,0,No,Hydrocodone,every 8 hours,1,0,Opioid,Moderate,Routine Check-up,480,Low,10
1,21,Female,Asian,Cancer Pain,11,70,25,12,0,Yes,Hydromorphone,every 8 hours,20,0,Narcotic,Low,Consultation,1540,Medium,100
2,67,Female,White,Fibromyalgia,5,15,7,10,2,No,Hydrocodone,every 4-6 hours,7,2,Analgesic,Moderate,Routine Check-up,75,Low,20
3,41,Male,White,Fibromyalgia,16,94,19,14,2,No,Oxymorphone,every 12 hours,28,0,Analgesic,Low,Consultation,3008,High,10
4,36,Male,White,Post-Surgery Pain,7,22,6,9,3,Yes,Tramadol,every 4-6 hours,25,2,Analgesic,High,Follow-up,154,Low,80


In [7]:
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, LabelEncoder
# Define the order for the ordinal categories
frequency_order = ['every 8 hours', 'every 4-6 hours', 'every 12 hours', 'once daily']
adherence_order = ['Low', 'Moderate', 'High']
# Ordinal Encoding for Frequency and Adherence
ordinal_columns = ['Frequency', 'Adherence']

ordinal_encoder = OrdinalEncoder(categories=[frequency_order, adherence_order])
data[ordinal_columns] = ordinal_encoder.fit_transform(data[ordinal_columns])
data.head()

,Age,Gender,Race,ChronicPainConditions,NumOpioidPrescriptions,AverageDosage,DurationOfPrescriptions,NumHealthcareVisits,NumHospitalizations,PainManagementTreatment,MedicationName,Frequency,Duration,Refills,MedicationClass,Adherence,AppointmentType,RiskScore,RiskLevel,Dosage_numeric
0,54,Female,Other,Fibromyalgia,6,80,7,18,0,No,Hydrocodone,0.0,1,0,Opioid,1.0,Routine Check-up,480,Low,10
1,21,Female,Asian,Cancer Pain,11,70,25,12,0,Yes,Hydromorphone,0.0,20,0,Narcotic,0.0,Consultation,1540,Medium,100
2,67,Female,White,Fibromyalgia,5,15,7,10,2,No,Hydrocodone,1.0,7,2,Analgesic,1.0,Routine Check-up,75,Low,20
3,41,Male,White,Fibromyalgia,16,94,19,14,2,No,Oxymorphone,2.0,28,0,Analgesic,0.0,Consultation,3008,High,10
4,36,Male,White,Post-Surgery Pain,7,22,6,9,3,Yes,Tramadol,1.0,25,2,Analgesic,2.0,Follow-up,154,Low,80


In [8]:
# Label Encoding for PainManagementTreatment
label_encoder = LabelEncoder()
data['PainManagementTreatment'] = label_encoder.fit_transform(data['PainManagementTreatment'])

In [9]:
data.head()

,Age,Gender,Race,ChronicPainConditions,NumOpioidPrescriptions,AverageDosage,DurationOfPrescriptions,NumHealthcareVisits,NumHospitalizations,PainManagementTreatment,MedicationName,Frequency,Duration,Refills,MedicationClass,Adherence,AppointmentType,RiskScore,RiskLevel,Dosage_numeric
0,54,Female,Other,Fibromyalgia,6,80,7,18,0,0,Hydrocodone,0.0,1,0,Opioid,1.0,Routine Check-up,480,Low,10
1,21,Female,Asian,Cancer Pain,11,70,25,12,0,1,Hydromorphone,0.0,20,0,Narcotic,0.0,Consultation,1540,Medium,100
2,67,Female,White,Fibromyalgia,5,15,7,10,2,0,Hydrocodone,1.0,7,2,Analgesic,1.0,Routine Check-up,75,Low,20
3,41,Male,White,Fibromyalgia,16,94,19,14,2,0,Oxymorphone,2.0,28,0,Analgesic,0.0,Consultation,3008,High,10
4,36,Male,White,Post-Surgery Pain,7,22,6,9,3,1,Tramadol,1.0,25,2,Analgesic,2.0,Follow-up,154,Low,80


In [10]:
# Encoding
# One-Hot Encoding for Gender, Race, ChronicPainConditions, MedicationName, Dosage, MedicationClass, AppointmentType
one_hot_columns = ['Gender', 'Race', 'ChronicPainConditions', 'MedicationName', 'MedicationClass', 'AppointmentType']
# Initialize the LabelEncoder
label_encoders = {}

# Apply LabelEncoder to each column
for col in one_hot_columns:
    le = LabelEncoder()
    data[col] = le.fit_transform(data[col])
    label_encoders[col] = le

# Check the transformed data
data.head()

,Age,Gender,Race,ChronicPainConditions,NumOpioidPrescriptions,AverageDosage,DurationOfPrescriptions,NumHealthcareVisits,NumHospitalizations,PainManagementTreatment,MedicationName,Frequency,Duration,Refills,MedicationClass,Adherence,AppointmentType,RiskScore,RiskLevel,Dosage_numeric
0,54,0,3,3,6,80,7,18,0,0,3,0.0,1,0,2,1.0,2,480,Low,10
1,21,0,0,1,11,70,25,12,0,1,4,0.0,20,0,1,0.0,0,1540,Medium,100
2,67,0,4,3,5,15,7,10,2,0,3,1.0,7,2,0,1.0,2,75,Low,20
3,41,1,4,3,16,94,19,14,2,0,9,2.0,28,0,0,0.0,0,3008,High,10
4,36,1,4,4,7,22,6,9,3,1,11,1.0,25,2,0,2.0,1,154,Low,80


In [11]:
# Define the mapping from categorical to numerical values
label_mapping = {'Low': 0, 'Medium': 1, 'High': 2}

# Assuming your target variable column is named 'Target'
data['RiskLevel'] = data['RiskLevel'].map(label_mapping)

# Print the first few rows to check the conversion
data.head()

,Age,Gender,Race,ChronicPainConditions,NumOpioidPrescriptions,AverageDosage,DurationOfPrescriptions,NumHealthcareVisits,NumHospitalizations,PainManagementTreatment,MedicationName,Frequency,Duration,Refills,MedicationClass,Adherence,AppointmentType,RiskScore,RiskLevel,Dosage_numeric
0,54,0,3,3,6,80,7,18,0,0,3,0.0,1,0,2,1.0,2,480,0,10
1,21,0,0,1,11,70,25,12,0,1,4,0.0,20,0,1,0.0,0,1540,1,100
2,67,0,4,3,5,15,7,10,2,0,3,1.0,7,2,0,1.0,2,75,0,20
3,41,1,4,3,16,94,19,14,2,0,9,2.0,28,0,0,0.0,0,3008,2,10
4,36,1,4,4,7,22,6,9,3,1,11,1.0,25,2,0,2.0,1,154,0,80


In [12]:
data.to_csv('preprocess2.csv', index=False)